# 03 fastTextのword embeddingsを使ってみる
* このnotebookは、Google Colabではなく、手元の環境で動かすことを想定しています。
 * Google Colabで動かすとかなり時間がかかると思います。
* 必要ならば、このnotebookを実行する前に、condaの環境を作っておきましょう。

`$ conda create -n D_wordvec`

`$ source activate D_wordvec`

## 03-01 fastTextをインストールする

In [1]:
!pip install fasttext

### "Word vectors for 157 languages"から英語データをダウンロード
* fastTextのドキュメント https://fasttext.cc/docs/en/crawl-vectors.html
* 論文 https://arxiv.org/abs/1802.06893

In [2]:
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

## 03-02 IMDbデータセットをダウンロード

### 本家のサイトからダウンロード
* 方法は他にもあるが、ここでは本家サイトから直にダウンロードする。

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-08-20 17:19:37--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
ai.stanford.edu (ai.stanford.edu) をDNSに問いあわせています... 171.64.68.10
ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 84125825 (80M) [application/x-gzip]
`aclImdb_v1.tar.gz.1' に保存中

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  15.2MB/s 時間 9.9s       

2020-08-20 17:19:47 (8.14 MB/s) - `aclImdb_v1.tar.gz.1' へ保存完了 [84125825/84125825]



In [4]:
!tar zxf aclImdb_v1.tar.gz

### ml-datasetsをインストール
* https://pypi.org/project/ml-datasets/
* 機械学習のデータセットのローダ。IMDbも簡単に扱える。

In [5]:
!pip install ml-datasets

### fastTextの単語ベクトルを読み込む
* さきほどダウンロードし、解凍しておいたものを読み込む。

In [6]:
import fasttext

model_path = 'cc.en.300.bin'
print(f'# loading {model_path} ...', flush=True) 
ft = fasttext.load_model(model_path)

# loading cc.en.300.bin ...


### IMDbデータセットを読み込む
* 本家サイトからダウンロードし、解凍しておいたものを、ml_datasetsを使って読み込む。

In [7]:
from ml_datasets import imdb

train_valid_data, test_data = imdb(loc='./aclImdb')

In [8]:
train_valid_data[:3]

[('Some people say Steve Irwin\'s larrikin antics and gregarious personality are only an act. Watch this film: it\'s obvious he can\'t act.\n\n\n\nSteve Irwin, dangerman star of the small screen in his *Crocodile Hunter Diaries*, *Croc Files* and eponymous *Crocodile Hunter* series (you see a naming trend here, or is it just me?), rockets his larger-than-strife persona to the big screen with *Crocodile Hunter: Collision Course* (yup \x96 there\'s a definite trend of words beginning with \'C\') - basically an episode of *Crocodile Hunter* mashed together with a B-Movie.\n\n\n\nOn a mission to relocate a big croc to save it from being shot by an eccentric farmer (Magda Szubanski), Steve and wife Terri are unaware that the croc is being tracked by American spies (Lachy Hulme and Kenneth Ransom), out to recover a spy satellite beacon it has swallowed. Will it hurt my credibility to say "They\'re on a Collision Course with Wackiness"? (what credibility? - Ed note.)\n\n\n\nThe plot is irrele

### テキスト部分と0/1ラベル部分に分ける

In [9]:
train_valid_texts, train_valid_labels = zip(*train_valid_data)
test_texts, test_labels = zip(*test_data)

### テストセット以外をランダムにシャッフル

In [10]:
import random

random.seed(123)
random.shuffle(train_valid_data)

### 手動で訓練データと検証データへ分割

In [11]:
split = int(len(train_valid_data) * 0.8)
train_texts, train_labels = train_valid_texts[:split], train_valid_labels[:split]
valid_texts, valid_labels = train_valid_texts[split:], train_valid_labels[split:]

In [12]:
print(f'# {len(train_texts)} training, {len(valid_texts)} validation, and {len(test_texts)} test docs')

# 20000 training, 5000 validation, and 25000 test docs


In [13]:
splits = {
    'train': (train_texts, train_labels),
    'valid': (valid_texts, valid_labels),
    'test': (test_texts, test_labels)
}

### 全文書のembeddingを得てファイルに保存
* fastTextのget_sentence_vectorを使って文書のベクトル表現を得る。
* 全文書のベクトル表現をndarrayに変換、`.npy`形式で保存
* 全文書のラベルもndarrayに変換、`.npy`形式で保存

In [15]:
import numpy as np

for tag in splits:
    print(f'# {tag} set: ', end='', flush=True)
    cnt = 0
    X = list()
    for text in splits[tag][0]:
        vec = ft.get_sentence_vector(' '.join(text.split('\n')))
        X.append(vec)
        cnt += 1
        if cnt % 10000 == 0: print('*', end='', flush=True)
        elif cnt % 1000 == 0: print('-', end='', flush=True)
    X = np.array(X)
    with open(f'{tag}.npy', 'wb') as f:
        np.save(f, X, allow_pickle=False)
    with open(f'{tag}_labels.npy', 'wb') as f:
        np.save(f, np.array(splits[tag][1]), allow_pickle=False)
    print(flush=True)

# train set: ---------*---------*
# valid set: -----
# test set: ---------*---------*-----


In [16]:
!ls -al *.npy

-rw-r--r--  1 masada  staff  30000128  8 20 17:26 test.npy
-rw-r--r--  1 masada  staff    200128  8 20 17:26 test_labels.npy
-rw-r--r--  1 masada  staff  24000128  8 20 17:26 train.npy
-rw-r--r--  1 masada  staff    160128  8 20 17:26 train_labels.npy
-rw-r--r--  1 masada  staff   6000128  8 20 17:26 valid.npy
-rw-r--r--  1 masada  staff     40128  8 20 17:26 valid_labels.npy
